# Notebook to run gPhoton
Run gPhoton to create GALEX light curves etc.
Setup your source parameter and setting in the cell below.

In [1]:
from astropy.io import ascii
from astropy import units as uu

src_str = """label	ra	dec
WD900 215.3258439968967	52.596545351520945"""
tt_srcs = ascii.read(src_str)
outdir = "./resources/gPhoton_out/"
outfile_fin = outdir+"gPhoton_"+str(tt_srcs["label"][0])+'_fin.npy'
outfile_app = outdir+"gPhoton_"+str(tt_srcs["label"][0])+'_app.npy'
ron = (5*uu.arcsec).to(uu.deg).value        # Radius of signal annulus
roff1 = (17*uu.arcsec).to(uu.deg).value      # Inner radius of background ring
roff2 = (20*uu.arcsec).to(uu.deg).value      # Outer radius of background ring
#load_past_results = False

In [2]:
%matplotlib widget
import gPhoton
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import ascii
from astropy.table import Table
import os

### Do aperture photometry for each visit with gPhoton

In [ ]:
%%time
pos_ra = tt_srcs["ra"][0]
pos_dec = tt_srcs["dec"][0]

#Run or load gFind
if os.path.isfile(outfile_fin):
    dd_gfind = np.load(outfile_fin,allow_pickle='TRUE').item()
    t_bins = list(zip(dd_gfind["NUV"]["t0"],dd_gfind["NUV"]["t1"]))
    print(t_bins)
else:
    print(f"Running query with gFind for:\n",tt_srcs[0])
    dd_gfind = gPhoton.gFind(band='NUV',skypos=[pos_ra,pos_dec]) #,maxgap=100.,minexp=100.
    t_bins = list(zip(dd_gfind["NUV"]["t0"],dd_gfind["NUV"]["t1"]))
    np.save(outfile_fin,dd_gfind)

#Run or load gApperture
if os.path.isfile(outfile_app):
    dd_gph = np.load(outfile_app,allow_pickle='TRUE').item()
else:
    print("Running lightcurve with gAperture..")
    dd_gaperture = gPhoton.gAperture(band='NUV',skypos=[pos_ra,pos_dec],radius=ron, annulus=[roff1,roff2],tranges=t_bins)
    dd_gph = {"gAperture":dd_gaperture,"gFind":dd_gfind}
    np.save(outfile_app,dd_gph)
    print("..done")

[(740062663.995, 740064275.995), (740299455.995, 740300963.995), (740334978.995, 740336467.995), (740341003.995, 740342384.995), (740377425.995, 740377890.995), (740382345.995, 740383806.995), (740465247.995, 740466653.995), (740506701.995, 740508078.995), (740548158.995, 740549505.995), (740577772.995, 740579096.995), (740619520.995, 740620525.995), (740660697.995, 740661957.995), (740666620.995, 740667875.995), (740702239.995, 740703390.995), (740743645.995, 740744827.995), (740749553.995, 740750746.995), (740785099.995, 740786266.995), (740791024.995, 740792174.995), (764039467.995, 764039923.995), (764045385.995, 764045971.995), (764080888.995, 764082592.995), (764086805.995, 764088509.995), (771703452.995, 771704532.995), (771792694.995, 771793339.995), (771881126.995, 771882164.995), (771969925.995, 771971002.995), (794296814.995, 794298211.995), (794302731.995, 794304250.995), (794332317.995, 794333978.995), (794338233.995, 794339895.995), (794344151.995, 794345813.995), (794367

In [4]:
print(dd_gph["gAperture"].keys())
print("\n", dd_gph["gFind"]["NUV"].keys())
#print(dd_gph["gAperture"]['photons'])
keep_keys = ('t_mean', 'exptime','flux_bgsub', 'flux_bgsub_err','mag_bgsub', 'mag_bgsub_err_1', 'mag_bgsub_err_2',"flags")
dd_gap = {x: dd_gph["gAperture"][x] for x in keep_keys if x in dd_gph["gAperture"]}
tt_gap = Table(data= dd_gap)
tt_gap["s2n"]=dd_gap['flux_bgsub']/dd_gap['flux_bgsub_err']
sel_gap = tt_gap["s2n"]>3.5
display(tt_gap[sel_gap])

NameError: name 'dd_gph' is not defined

### Display lightcurve

In [ ]:
fig = plt.figure(figsize=(8, 6), constrained_layout=True)
plt.errorbar(tt_gap[sel_gap]['t_mean'],tt_gap[sel_gap]['flux_bgsub'],yerr=tt_gap[sel_gap]['flux_bgsub_err'],fmt="bo")

fig2 = plt.figure(figsize=(8, 6), constrained_layout=True)
ax = plt.gca()
ax.invert_yaxis()
plt.errorbar(tt_gap[sel_gap]['t_mean'],tt_gap[sel_gap]['mag_bgsub'],yerr=tt_gap[sel_gap]['mag_bgsub_err_1'],fmt="bo")